In [2]:
import yfinance as yf
import pandas as pd
from statsmodels.tsa.stattools import coint

tickers = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLV', 'XLY', 'XLU']
prices = yf.download(tickers, start = '2015-07-22', end = '2020-07-22')['Close']

returns = prices.pct_change().dropna()

correlation_matrix = returns.corr()
correlation_matrix = correlation_matrix.rename_axis(None).rename_axis(None, axis = 1)
correlation_matrix = correlation_matrix.stack().reset_index()
correlation_matrix.columns = ['ETF', 'Pair', 'Correlation']

correlation_matrix =  correlation_matrix[correlation_matrix['ETF'] != correlation_matrix['Pair']]
correlation_matrix = correlation_matrix.sort_values(by = 'Correlation', ascending = False)
correlation_matrix = correlation_matrix.drop_duplicates('Correlation')

potential_pairs = correlation_matrix[correlation_matrix['Correlation'] > 0.8]

potential_pairs

[*********************100%***********************]  9 of 9 completed


,ETF,Pair,Correlation
3,XLB,XLI,0.897353
29,XLI,XLF,0.895759
44,XLK,XLY,0.892242
75,XLY,XLI,0.863157
2,XLB,XLF,0.845627
74,XLY,XLF,0.828372
72,XLY,XLB,0.820797
31,XLI,XLK,0.816501
67,XLV,XLK,0.805193


In [19]:
tickers = prices.columns
cointegrated_pairs = []

for i in range(len(tickers)):
        for j in range(i + 1, len(tickers)):
                etf1 = tickers[i]
                etf2 = tickers[j]

                score, pvalue, _ = coint(prices[etf1], prices[etf2])

                if pvalue < 0.05:
                        cointegrated_pairs.append((etf1, etf2, pvalue))
                
cointegrated_pairs_df = pd.DataFrame(cointegrated_pairs, columns = ['ETF', 'Pair', 'P-Value'])
cointegrated_pairs_df = cointegrated_pairs_df.sort_values('P-Value').reset_index(drop=True)

print("Cointegration Test Results:")
cointegrated_pairs_df

Cointegration Test Results:


,ETF,Pair,P-Value
0,XLP,XLU,0.006064
1,XLK,XLV,0.013148
2,XLF,XLI,0.019970
